import pyarabic.araby as araby
import pyarabic.number as number
import pyarabic.named as named
#import normalize_hamza
txt=data['text']
txt[1].split()
txt[1].named
#normalize_hamza(txt[5])

In [54]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

wv = gensim.models.Word2Vec.load('models/full_grams_cbow_100_twitter.mdl')
from gensim.models import Word2Vec
#wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)

In [55]:
from itertools import islice
list(islice(wv.wv.vocab, 13030, 13050))

['تفائل',
 'وصباح',
 'حنه',
 'ورنه',
 'عارفين_نفسهم',
 'اثير',
 'اهوي',
 'المشاك',
 'تبتعد_عنه',
 'بالم',
 'قاع',
 'واححد',
 'ياورده',
 'اكممل',
 'وردده',
 'كييس',
 'مليان',
 'هاللعبه',
 'لول',
 'خادم']

In [56]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list])

In [57]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [58]:
import os, sys
import pandas as pd
import numpy as np
#readind and preparing data ( text , label )
df=pd.read_excel('ML1.xlsx', header=None) ## read le file
df.columns=['tags','post']
print(df[1:5])

   tags                                               post
1     3  محضر تحقيق بادعاء المدعوة اسما هدى انيس قساطلي...
2     4  محضر تحقيق بحق المدعو ماهر صالح الحرامي سوري ل...
3     1  محضر بتنفيذ خلاصة حكم بحق المدعو محمد سعد الدي...
4     2  محضر بتنفيذ قرار جزائي بحق المدعو محمد سعد الد...


In [59]:
train, test = train_test_split(df, test_size=0.3, random_state = 42)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['post']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['post']), axis=1).values

In [60]:
print(test_tokenized[1])

['محضر', 'تحقيق', 'بالشكوى', 'المقدمة', 'من', 'ليال', 'حماده', 'ضد', 'المؤهل', 'مهدي', 'ظنيط', 'في', 'الامن', 'العام', 'بجرم', 'استيلاء', 'على', 'سيارتها', 'تسليمها', 'تركه', 'سند', 'اقامة']


In [61]:
X_train_word_average = word_averaging_list(wv.wv,train_tokenized)
X_test_word_average = word_averaging_list(wv.wv,test_tokenized)

C:\Users\Maya\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  


In [62]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_word_average, train['tags'])
y_pred = logreg.predict(X_test_word_average)

C:\Users\Maya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Maya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [63]:
from sklearn.metrics import classification_report
print('accuracy %s' % accuracy_score(y_pred, test.tags))
print(classification_report(test.tags, y_pred))

accuracy 0.9175084175084175
              precision    recall  f1-score   support

           0       0.76      0.71      0.74        70
           1       0.96      0.98      0.97        94
           2       0.99      0.98      0.99       103
           3       0.96      0.95      0.96       110
           4       0.89      0.91      0.90       217

   micro avg       0.92      0.92      0.92       594
   macro avg       0.91      0.91      0.91       594
weighted avg       0.92      0.92      0.92       594



In [64]:
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text
def get_vec(n_model,dim, token):
    vec = np.zeros(dim)
    is_vec = False
    if token not in n_model.wv:
        _count = 0
        is_vec = True
        for w in token.split("_"):
            if w in n_model.wv:
                _count += 1
                vec += n_model.wv[w]
        if _count > 0:
            vec = vec / _count
    else:
        vec = n_model.wv[token]
    return vec

def calc_vec(pos_tokens, neg_tokens, n_model, dim):
    vec = np.zeros(dim)
    for p in pos_tokens:
        vec += get_vec(n_model,dim,p)
    for n in neg_tokens:
        vec -= get_vec(n_model,dim,n)
    
    return vec   


In [65]:
t_model = wv
pos_tokens=[ clean_str(t.strip()).replace(" ", "_") for t in ['نارية', 'سرقة'] if t.strip() != ""]
neg_tokens=[ clean_str(t.strip()).replace(" ", "_") for t in ['نشل'] if t.strip() != ""]

vec = calc_vec(pos_tokens=pos_tokens, neg_tokens=neg_tokens, n_model=t_model, dim=t_model.vector_size)

most_sims = t_model.wv.similar_by_vector(vec, topn=10)
for term, score in most_sims:
    if term not in pos_tokens+neg_tokens:
        print(term, score)


ملغومه 0.5874800682067871
ضخمه 0.5862234234809875
عشوائيه 0.5828266143798828
الناريه 0.5813839435577393
تاريخيه 0.5780879855155945
خطيره 0.5702124834060669
غير_مسبوقه 0.5676120519638062
مشبوهه 0.5671120882034302


In [66]:
token = clean_str(u'جرم')

most_similar = t_model.wv.most_similar( token, topn=10)
for term, score in most_similar:
    print(term, score)

ينهاكم 0.630063533782959
يصلح_عمل 0.5926780104637146
شر_الدواب_عند 0.5893514156341553
ما_قدروا 0.5666218996047974
تربحهم 0.5639942288398743
{كتب 0.5596617460250854
الله_ورسوله_واتقوا 0.5563033223152161
تعالي_يغار 0.5538311004638672
وما_يذكرون 0.5401697158813477
انما_نطعمكم_لوجه 0.5332059860229492
